In [1]:
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException
from animalai.envs.gym.environment import AnimalAIGym

In [6]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import os
import random
from ray.tune.registry import register_env
from ray.rllib.agents import ppo

In [16]:
class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('/home/azibit/Downloads/AnimalAI-Olympics/examples/configurations/curriculum/0.yml')
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [17]:
def env_creator(env_config):
    return UnityEnvWrapper(env_config)

In [19]:
ray.shutdown()
ray.init()
register_env("my_env", env_creator)
conf = {"env": "unity_env",
            "num_workers": 0,
            "env_config": {
                "unity_worker_id": 52
            },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }

trainer = ppo.PPOTrainer(env="my_env", config = conf)

2021-03-19 00:44:17,909	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=39414) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=39414) Instructions for updating:
(pid=39414) non-resource variables are not supported in the long term
(pid=39414) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=39414) Instructions for updating:
(pid=39414) non-resource variables are not supported in the long term
(pid=39414) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (fro

(pid=39414) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=39414) Mono path[0] = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/Managed'
(pid=39414) Mono config path = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=39414) Preloaded 'lib_burst_generated.so'
(pid=39414) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=39414) Preloaded 'ScreenSelector.so'
(pid=39414) Display 0 '0': 1920x1080 (primary device).
(pid=39414) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log
(pid=39414) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=39414) Mono path[0] = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/Managed'
(pid=39414) Mono config path = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=39414) Preloaded 'lib_burst_generated.so'
(pid=39414) Preloa

(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=39414) INFO:mlagents_envs:Connected to Unity environment with package version 

In [23]:
from ray.tune.logger import pretty_print

for i in range(5):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 5 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

custom_metrics: {}
date: 2021-03-19_00-54-08
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 7
experiment_id: 0db1d83a14d7435889de8e9e1b88d1ab
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.45000000000000007
      cur_lr: 5.0e-05
      entropy: 0.23966861367225648
      entropy_coeff: 0.0
      kl: 0.04264703951776028
      policy_loss: -0.05175026468932629
      total_loss: 0.29562865793704984
      vf_explained_var: 0.7067931890487671
      vf_loss: 0.32818777561187745
  num_steps_sampled: 1800
  num_steps_trained: 1800
iterations_since_restore: 3
node_ip: 10.29.158.247
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 15.415315315315315
  ram_util_percent: 48.11351351351352
pid: 36534
policy_reward_max: {}
policy_reward_mean: {}
policy_rew

In [24]:
trainer